## Geocoding Process

Cleans and processes input addresses before running through the Google Maps API


Input: street address line 1, city, state, and zip code
Output: Geocoded address, city, county, state, latitude, longitude, process status (Geocoded, No Address Found, No Address Provided)


Requires command line arguments in the following order: (1) server name, (2) database name, (3) source schema.tableName, (4) output folder name (for bulk insert), (5) output table name and (6) Google API key


In [1]:
import sys
import re
import pandas as pd
import requests
import numpy as np
import logging
import pyodbc

In [2]:
def udf_get_address_components_columns(df):
    
    '''
    Naively select the address columns
    
    Input: target dataframe
    Output: list of address components
    '''
    
    l_address = []
    l_city = []
    l_state = []
    l_zip = []
    for col in df.columns:
        if 'Address'.upper() in col.upper():
            l_address.append(col)
        elif 'City'.upper() in col.upper():
            l_city.append(col)
        elif 'State'.upper() in col.upper():
            l_state.append(col)
        elif 'Zip'.upper() in col.upper():
            l_zip.append(col)
    
    l_address.sort()
    address = l_address[0]
    
    l_city.sort()
    city = l_city[0]

    l_state.sort()
    state = l_state[0]
    
    l_zip.sort()
    zipcode = l_zip[0]
    
    address_components = list([address, city, state, zipcode])
    
    return address_components

In [3]:
def udf_remove_address_unit(address_str):
    
    '''
    Pre-process address by capturing address line 2 references
    Input: address as a string
    Output: processed address
    
    Remove any housing unit references from the address and ensure the last character of the address is a letter
    '''
    
    # Remove Unit Name
    address_str = str(address_str)
    address_processed = address_str.upper()
    address_processed = address_processed.partition("APT")[0]
    address_processed = address_processed.partition("UNIT")[0]
    address_processed = address_processed.partition("SUITE")[0]
    address_processed = address_processed.partition("STE")[0]
    
    # remove all non-alphanumeric characters at the end of the address
    address_processed = re.sub(f'[^1-9a-zA-Z]+$','',address_processed)
    
    return address_processed

In [29]:
'''
Google service not handling bad street numbers (ex: 6975A SPRINGFIELD) and bad street names (ex: JOHN TAYLOR PARKWAY Williamsburg VA 23187)
Need function to clean data
'''

'\nGoogle service not handling bad street numbers (ex: 6975A SPRINGFIELD) and bad street names (ex: JOHN TAYLOR PARKWAY Williamsburg VA 23187)\nNeed function to clean data\n'

In [25]:
def udf_get_google_geocoded_address (address_str, api_key):
    
    # Address Components
    address_str_geocoded = np.NaN
    street_number = np.NaN
    street_name = np.NaN
    city = np.NaN
    county = np.NaN
    state = np.NaN
    zipcode = np.NaN
    latitude = np.NaN
    longitude = np.NaN
    
    # process populated addresses
    if pd.isnull(address_str) == False:
        geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address_str)
        geocode_url = geocode_url + "&key={}".format(api_key)

        results = requests.get(geocode_url)
        results = results.json()
        results = results['results']
       
        # Check if address is valid, update address components
        if (len(results) > 0):
    
            results = results[0]

            for i in range(len(results.get('address_components'))):
                if results.get('address_components')[i].get('types')[0]=='street_number':
                    street_number = results.get('address_components')[i].get('long_name')
                elif results.get('address_components')[i].get('types')[0]=='route':
                    street_name = results.get('address_components')[i].get('long_name')
                elif results.get('address_components')[i].get('types')[0]=='locality':
                    city = results.get('address_components')[i].get('long_name')
                elif results.get('address_components')[i].get('types')[0]=='administrative_area_level_2':
                    county = results.get('address_components')[i].get('long_name')
                elif results.get('address_components')[i].get('types')[0]=='administrative_area_level_1':
                    state = results.get('address_components')[i].get('long_name')
                elif results.get('address_components')[i].get('types')[0]=='postal_code':
                    zipcode = results.get('address_components')[i].get('long_name')
                
            address_str_geocoded = street_number + ' ' + street_name
                
            latitude = results.get('geometry').get('location')['lat']
            longitude = results.get('geometry').get('location')['lng']
            
            status = 'Geocoded'
            
        else:
            status = 'Address Not Found'
    
    else:
        status = 'No Address Provided'
    
    
    output = {
        'Address_Geocoded' : address_str_geocoded,
        'City_Geocoded' : city,
        'County_Geocoded' : county,
        'State_Geocoded' : state,
        'ZipCode_Geocoded' : zipcode,
        'Latitude' : latitude,
        'Longitude' : longitude,
        'Address_Full_PreProcessed' : address_str,
        'Status_Geocoded' : status
        }
    
    return output

In [27]:
def udf_geocode_addresses(df, api_key):
    
    '''
    Given a dataframe, df, of addresses, and a Google API Key
        return a data frame of geocoded addresses
    '''
    
    # get address components of df
    address_components_col = udf_get_address_components_columns(df)
    address_col = address_components_col[0]
    city_col = address_components_col[1]
    state_col = address_components_col[2]
    zipcode_col = address_components_col[3]
    
    df_geocoded_addresses = pd.DataFrame()
    
    df[(address_col+'_PreProcessed')] = df[address_col].apply(udf_remove_address_unit)

    df[(address_col+'_Full_PreProcessed')] = df[(address_col+'_PreProcessed')] + ' ' + df[city_col] + ' ' + df[state_col] + ' ' + df[zipcode_col]
    
    df_log_address_errors = pd.DataFrame(columns=df.columns)
    
    for index, row in df.iterrows():
        try:
            df_geocoded_addresses = df_geocoded_addresses.append(udf_get_google_geocoded_address(row[(address_col+'_Full_PreProcessed')],api_key),ignore_index=True)
        except:
            df_log_address_errors = df_log_address_errors.append(pd.DataFrame(row),ignore_index=True)
            print('ERROR: %s' % index)

    return df_geocoded_addresses

In [6]:
# Write to Database via bulk insert
def udf_bulk_insert_db(df, server_name, db_name, target_table, output_file_path, output_file_name):
    
    # write file to output_folder_path
    df.to_csv(output_file_path + output_file_name,index=False)
    
    # insert into target table
    conn = pyodbc.connect(driver="{SQL Server}", server=server_name, Database=db_name, trusted_connection="yes")
    cursor = conn.cursor()
    sql = "BULK INSERT " + target_table + " FROM '" + output_file_path + output_file_name + "' WITH (FIRSTROW = 2, FORMAT = 'CSV');"  
    cursor.execute(sql)
    conn.commit()
    cursor.close()


In [7]:
# Get command line arguments
cmd_line_args = sys.argv[1].split(sep=" ")

server_name = str(cmd_line_args[0]).replace('\'','')
db_name = str(cmd_line_args[1]).replace('\'','')
table_name = str(cmd_line_args[2]).replace('\'','')
output_file_name = table_name.split('.')[1] + '_Geocoded.csv'
output_file_path = str(cmd_line_args[3]).replace('\'','')
output_table_name = str(cmd_line_args[4]).replace('\'','')
api_key = str(cmd_line_args[5]).replace('\'','')

# Get data
conn = pyodbc.connect(driver="{SQL Server}", server=server_name, Database=db_name, trusted_connection="yes")
sql = 'Select * From ' + table_name + ' '
df = pd.read_sql(sql, conn, chunksize=10000)
df_addresses = pd.DataFrame()
df_addresses = pd.concat(df)
conn.close()



In [29]:
# Geocode addresses
df_addresses_geocoded = udf_geocode_addresses(df_addresses, api_key)

<class 'pandas.core.series.Series'>
ERROR: 64
<class 'pandas.core.series.Series'>
ERROR: 108
<class 'pandas.core.series.Series'>
ERROR: 126
<class 'pandas.core.series.Series'>
ERROR: 137
<class 'pandas.core.series.Series'>
ERROR: 164
<class 'pandas.core.series.Series'>
ERROR: 184
<class 'pandas.core.series.Series'>
ERROR: 192
<class 'pandas.core.series.Series'>
ERROR: 197
<class 'pandas.core.series.Series'>
ERROR: 221
<class 'pandas.core.series.Series'>
ERROR: 239


In [ ]:
# Merge back to original data frame on (address+'_PreProcessed')
# Drop '_PreProcessed' column
# Split out line 2?

# Add process date


In [ ]:
# Load Data
udf_bulk_insert_db(df_addresses, server_name, db_name, target_table, output_file_path, output_file_name)

# Delete file from output folder
os.remove(output_file_path + output_file_name)